# Personalized Financial Advisor

This interactive notebook demonstrates how language models can function as personalized financial advisors, adapting explanations to users' knowledge levels.

## What is a Personalized Financial Advisor?

A personalized financial advisor in the context of AI provides customized financial guidance based on an individual's specific situation, goals, and level of financial literacy. The key innovation is dynamic adaptation - the system adjusts its language, explanations, and recommendations based on the user's knowledge level.


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, HTML, clear_output, Markdown
import ipywidgets as widgets
import random
import os
from dotenv import load_dotenv
from openai import OpenAI
import sklearn
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import pipeline
import torch
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Set styling
plt.style.use("ggplot")
sns.set_palette("viridis")

# Suppress warnings
import warnings

warnings.filterwarnings("ignore")

# Load environment variables for API keys
load_dotenv()

# Download NLTK resources if needed
try:
    nltk.data.find("vader_lexicon")
except LookupError:
    nltk.download("vader_lexicon")

# Initialize OpenAI client
try:
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    has_openai_key = True
except:
    has_openai_key = False
    print("OpenAI API key not found. Some features will be disabled.")

# Initialize sentiment analyzer
sentiment_analyzer = SentimentIntensityAnalyzer()

OpenAI API key not found. Some features will be disabled.


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\super\AppData\Roaming\nltk_data...


## Financial Knowledge Level Adaptation

One of the key features of our personalized financial advisor is its ability to adapt explanations to the user's level of financial knowledge.


In [6]:
# Sample financial product explanations at different knowledge levels
financial_products = {
    "Mutual Funds": {
        "beginner": """
        **Mutual Funds - Beginner Explanation**
        
        Think of a mutual fund like a big basket of investments that many people put money into together. A professional manager takes all this pooled money and buys stocks, bonds, or other investments.
        
        **Key points:**
        - You buy "shares" of the fund (like pieces of the basket)
        - Your money is combined with other investors' money
        - Professional managers make the investment decisions for you
        """,
        "intermediate": """
        **Mutual Funds - Intermediate Explanation**
        
        Mutual funds are pooled investment vehicles that collect capital from many investors to purchase a diversified portfolio of securities, typically managed by professional fund managers according to a specific investment objective.
        
        **Structure and operations:**
        - Open-end funds continuously issue and redeem shares at Net Asset Value (NAV)
        - Closed-end funds issue a fixed number of shares that trade on exchanges
        - The fund's NAV is calculated daily based on closing market prices
        """,
        "advanced": """
        **Mutual Funds - Advanced Explanation**
        
        Mutual funds represent a mature vehicle in the asset management industry, operating under the Investment Company Act of 1940 with specific regulatory requirements regarding disclosure, diversification, and governance.
        
        **Advanced structural characteristics:**
        - Registered Investment Companies (RICs) must distribute at least 90% of investment income to maintain pass-through tax status
        - The '40 Act requires diversification with limits on position concentration (generally 5/10/25/50 rule)
        - Board governance with independent directors provides oversight on advisory contracts and fee structures
        """,
    },
    "Stocks": {
        "beginner": """
        **Stocks - Beginner Explanation**
        
        When you buy a stock, you're buying a small piece (share) of ownership in a company. As a shareholder, you participate in the company's success or failure.
        
        **Key points:**
        - Stock prices go up and down based on company performance and market conditions
        - You can make money through price increases (capital gains) and dividends
        - Generally higher risk than bonds, but with potential for higher returns
        """,
        "intermediate": """
        **Stocks - Intermediate Explanation**
        
        Stocks, or equities, represent ownership shares in corporations, entitling holders to a claim on assets and earnings. As residual claimants, stockholders have potential for unlimited upside but also stand last in line during liquidation.
        
        **Stock classifications:**
        - Market capitalization: large-cap, mid-cap, small-cap, micro-cap
        - Style: growth vs. value
        - Sectors and industries (using GICS or other classification systems)
        """,
        "advanced": """
        **Stocks - Advanced Explanation**
        
        Equity securities represent complex financial instruments where valuation is influenced by multiple factors including cash flow expectations, discount rates, market sentiment, and option-like characteristics inherent in the corporate structure.
        
        **Advanced valuation methodologies:**
        - Discounted cash flow using multi-stage models with terminal value calculations
        - Residual income models incorporating return on invested capital (ROIC)
        - Adjusted present value (APV) for companies with changing capital structures
        """,
    },
    "Bonds": {
        "beginner": """
        **Bonds - Beginner Explanation**
        
        A bond is like an IOU or a loan that you give to a company or government. In return, they promise to pay you interest regularly and return your original money (principal) when the bond matures (ends).
        
        **Key points:**
        - You're the lender, the bond issuer is the borrower
        - Bonds typically pay interest at a fixed rate (called the "coupon")
        - Generally considered safer than stocks, but offer lower potential returns
        """,
        "intermediate": """
        **Bonds - Intermediate Explanation**
        
        Bonds are debt securities where the issuer is obligated to pay the holder periodic interest payments and return the principal at maturity. The bond market is significantly larger than the equity markets globally.
        
        **Bond characteristics:**
        - Face value/par value is typically $1,000 for corporate bonds
        - Coupon rate represents annual interest as a percentage of face value
        - Yield to maturity (YTM) represents the total return if held to maturity
        """,
        "advanced": """
        **Bonds - Advanced Explanation**
        
        Fixed income securities represent a complex asset class with multifaceted risks and pricing models that incorporate term structure, credit spreads, and optionality considerations.
        
        **Advanced bond characteristics:**
        - Duration decomposition: modified duration, effective duration, key rate duration
        - Convexity as a second-order effect on price sensitivity
        - Option-adjusted spread (OAS) for callable and putable securities
        """,
    },
}

# Create an interface to explore financial products
knowledge_level = widgets.RadioButtons(
    options=["Beginner", "Intermediate", "Advanced"],
    value="Beginner",
    description="Knowledge Level:",
    style={"description_width": "initial"},
)

product_dropdown = widgets.Dropdown(
    options=list(financial_products.keys()),
    value=list(financial_products.keys())[0],
    description="Financial Product:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="50%"),
)

explanation_output = widgets.Output()


def update_explanation(*args):
    with explanation_output:
        clear_output()
        product = product_dropdown.value
        level = knowledge_level.value.lower()
        display(Markdown(financial_products[product][level]))


knowledge_level.observe(update_explanation, "value")
product_dropdown.observe(update_explanation, "value")

# Display the widgets
display(widgets.HTML("<h2>Financial Product Explorer</h2>"))
display(
    widgets.HTML(
        "<p>Select your knowledge level and a financial product to get a personalized explanation.</p>"
    )
)
display(widgets.VBox([widgets.HBox([knowledge_level]), product_dropdown, explanation_output]))

# Initial display
update_explanation()

HTML(value='<h2>Financial Product Explorer</h2>')

HTML(value='<p>Select your knowledge level and a financial product to get a personalized explanation.</p>')

## Risk and Return Visualization

Understanding the relationship between risk and return is fundamental to making informed investment decisions.


In [7]:
# Create risk-return data for different asset classes
asset_classes = {
    "Cash & Equivalents": {
        "return": 1.5,
        "risk": 0.5,
        "description": "Money market funds, Treasury bills, short-term CDs",
    },
    "Government Bonds": {
        "return": 3.0,
        "risk": 4.0,
        "description": "Treasury bonds, government agency bonds",
    },
    "Corporate Bonds (Investment Grade)": {
        "return": 4.5,
        "risk": 5.5,
        "description": "Bonds from large, stable companies with good credit ratings",
    },
    "Large-Cap Stocks": {
        "return": 8.5,
        "risk": 15.0,
        "description": "Stocks of large, established companies (e.g., S&P 500)",
    },
    "Small-Cap Stocks": {
        "return": 10.5,
        "risk": 22.0,
        "description": "Smaller companies with high growth potential",
    },
    "REITs": {"return": 7.5, "risk": 16.0, "description": "Real Estate Investment Trusts"},
    "Emerging Market Stocks": {
        "return": 11.0,
        "risk": 25.0,
        "description": "Stocks from developing economies (e.g., Brazil, India)",
    },
}

# Create dataframe for plotting
risk_return_df = pd.DataFrame(
    {
        "Asset Class": list(asset_classes.keys()),
        "Expected Return (%)": [asset_classes[ac]["return"] for ac in asset_classes],
        "Risk (Volatility %)": [asset_classes[ac]["risk"] for ac in asset_classes],
        "Description": [asset_classes[ac]["description"] for ac in asset_classes],
    }
)

# Create the scatter plot with Plotly
fig = px.scatter(
    risk_return_df,
    x="Risk (Volatility %)",
    y="Expected Return (%)",
    text="Asset Class",
    size=[10] * len(risk_return_df),  # Fixed size for all points
    hover_data=["Description"],
    title="Risk vs. Return by Asset Class",
    color="Expected Return (%)",
    color_continuous_scale="Viridis",
)

# Add efficient frontier curve (simplified approximation)
x_curve = np.linspace(0, max(risk_return_df["Risk (Volatility %)"]) * 1.1, 100)
y_curve = 1.5 + 2.5 * np.sqrt(x_curve)  # Simplified curve formula

fig.add_trace(
    go.Scatter(
        x=x_curve,
        y=y_curve,
        mode="lines",
        line=dict(color="rgba(100, 100, 100, 0.4)", dash="dash"),
        name="Efficient Frontier (approximation)",
    )
)

# Customize layout
fig.update_traces(textposition="top center")
fig.update_layout(
    height=500, xaxis_title="Risk (Standard Deviation %)", yaxis_title="Expected Annual Return (%)"
)

fig.show()

## Simple Portfolio Allocation Simulator

Let's create a basic tool to help users understand different portfolio allocations.


In [8]:
# Define portfolio allocation presets
portfolio_presets = {
    "Conservative": {
        "Cash": 10,
        "Bonds": 60,
        "Stocks (US)": 20,
        "Stocks (International)": 5,
        "Alternative Investments": 5,
    },
    "Moderate": {
        "Cash": 5,
        "Bonds": 40,
        "Stocks (US)": 35,
        "Stocks (International)": 15,
        "Alternative Investments": 5,
    },
    "Balanced": {
        "Cash": 5,
        "Bonds": 30,
        "Stocks (US)": 40,
        "Stocks (International)": 20,
        "Alternative Investments": 5,
    },
    "Growth": {
        "Cash": 5,
        "Bonds": 15,
        "Stocks (US)": 50,
        "Stocks (International)": 20,
        "Alternative Investments": 10,
    },
    "Aggressive Growth": {
        "Cash": 5,
        "Bonds": 5,
        "Stocks (US)": 55,
        "Stocks (International)": 25,
        "Alternative Investments": 10,
    },
}

# Portfolio preset dropdown
preset_dropdown = widgets.Dropdown(
    options=list(portfolio_presets.keys()),
    value="Balanced",
    description="Portfolio Type:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="50%"),
)

# Output area for results
portfolio_output = widgets.Output()


def update_portfolio(*args):
    with portfolio_output:
        clear_output()
        preset = preset_dropdown.value
        allocation = portfolio_presets[preset]

        # Create allocation pie chart
        fig = go.Figure()
        fig.add_trace(
            go.Pie(
                labels=list(allocation.keys()),
                values=list(allocation.values()),
                textinfo="label+percent",
                hole=0.3,
                marker=dict(colors=px.colors.qualitative.Plotly),
            )
        )
        fig.update_layout(title=f"{preset} Portfolio Allocation", height=400)
        fig.show()

        # Display expected return and risk
        # Simple approximation based on preset type
        risk_levels = {
            "Conservative": 5.0,
            "Moderate": 8.0,
            "Balanced": 10.0,
            "Growth": 13.0,
            "Aggressive Growth": 16.0,
        }

        return_levels = {
            "Conservative": 4.0,
            "Moderate": 5.5,
            "Balanced": 6.5,
            "Growth": 7.5,
            "Aggressive Growth": 8.5,
        }

        display(HTML(f"<h3>Portfolio Characteristics</h3>"))
        display(HTML(f"<p><b>Expected Annual Return:</b> {return_levels[preset]:.2f}%</p>"))
        display(HTML(f"<p><b>Expected Volatility:</b> {risk_levels[preset]:.2f}%</p>"))


# Connect preset dropdown to update function
preset_dropdown.observe(update_portfolio, "value")

# Display interface
display(widgets.HTML("<h2>Portfolio Allocation Explorer</h2>"))
display(widgets.HTML("<p>Select a portfolio type to see the typical allocation:</p>"))
display(widgets.VBox([preset_dropdown, portfolio_output]))

# Initial display
update_portfolio()

HTML(value='<h2>Portfolio Allocation Explorer</h2>')

HTML(value='<p>Select a portfolio type to see the typical allocation:</p>')

## Financial Sentiment Analysis

Understanding the sentiment behind financial news and reports is crucial for making informed investment decisions. Here we demonstrate how AI can analyze sentiment in financial text.


In [9]:
# Sample financial news headlines and statements
financial_texts = [
    "Company XYZ reports record profits for Q3, exceeding analyst expectations.",
    "Markets tumble amid concerns over rising inflation and interest rates.",
    "Tech sector faces regulatory challenges as new antitrust laws are proposed.",
    "Renewable energy stocks surge following new climate policy announcements.",
    "Banking sector shows resilience despite economic uncertainty.",
    "Consumer spending decreases for the third consecutive month.",
    "Manufacturing output falls short of projections, indicating potential slowdown.",
    "Startup raises $50M in Series B funding to expand operations globally.",
]

# Create an interface for text sentiment analysis
text_input = widgets.Textarea(
    value=financial_texts[0],
    placeholder="Enter financial text to analyze...",
    description="Text:",
    layout=widgets.Layout(width="90%", height="100px"),
)

example_dropdown = widgets.Dropdown(
    options=financial_texts,
    value=financial_texts[0],
    description="Examples:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="90%"),
)

analyze_button = widgets.Button(
    description="Analyze Sentiment", button_style="primary", layout=widgets.Layout(width="200px")
)

sentiment_output = widgets.Output()


def update_text_input(change):
    text_input.value = change.new


def analyze_sentiment(b):
    with sentiment_output:
        clear_output()
        text = text_input.value

        if not text.strip():
            print("Please enter some text to analyze.")
            return

        # Get sentiment scores using VADER
        sentiment_scores = sentiment_analyzer.polarity_scores(text)
        compound_score = sentiment_scores["compound"]

        # Determine sentiment category
        if compound_score >= 0.05:
            sentiment = "Positive"
            color = "green"
        elif compound_score <= -0.05:
            sentiment = "Negative"
            color = "red"
        else:
            sentiment = "Neutral"
            color = "gray"

        # Create visualization
        fig = go.Figure()

        # Add sentiment score gauge
        fig.add_trace(
            go.Indicator(
                mode="gauge+number",
                value=compound_score * 100,  # Scale to -100 to 100
                title={"text": "Sentiment Score"},
                gauge={
                    "axis": {"range": [-100, 100]},
                    "bar": {"color": color},
                    "steps": [
                        {"range": [-100, -5], "color": "lightcoral"},
                        {"range": [-5, 5], "color": "lightgray"},
                        {"range": [5, 100], "color": "lightgreen"},
                    ],
                    "threshold": {
                        "line": {"color": "black", "width": 3},
                        "thickness": 0.75,
                        "value": compound_score * 100,
                    },
                },
            )
        )

        fig.update_layout(height=300, width=600, margin=dict(l=50, r=50, t=30, b=30))

        # Display results
        display(
            HTML(
                f"<h3>Sentiment Analysis Result: <span style='color:{color}'>{sentiment}</span></h3>"
            )
        )
        display(HTML(f"<p><b>Text analyzed:</b> {text}</p>"))
        display(HTML("<p><b>Detailed scores:</b></p>"))
        display(pd.DataFrame([sentiment_scores]))
        fig.show()

        # Investment implication (simplified example)
        if sentiment == "Positive":
            implication = "This positive sentiment may indicate favorable market conditions or company performance."
        elif sentiment == "Negative":
            implication = "This negative sentiment may signal potential risks or market concerns."
        else:
            implication = "The neutral sentiment suggests balanced market conditions without strong signals in either direction."

        display(HTML(f"<p><b>Potential investment implication:</b> {implication}</p>"))


example_dropdown.observe(update_text_input, "value")
analyze_button.on_click(analyze_sentiment)

# Display the widgets
display(widgets.HTML("<h2>Financial Text Sentiment Analyzer</h2>"))
display(
    widgets.HTML(
        "<p>Analyze the sentiment of financial news and statements to gauge market sentiment.</p>"
    )
)
display(
    widgets.VBox(
        [
            widgets.HTML("<p><b>Select from examples or enter your own text:</b></p>"),
            example_dropdown,
            text_input,
            analyze_button,
            sentiment_output,
        ]
    )
)

# Initial analysis
analyze_sentiment(None)

HTML(value='<h2>Financial Text Sentiment Analyzer</h2>')

HTML(value='<p>Analyze the sentiment of financial news and statements to gauge market sentiment.</p>')

## Personalized Financial Advice with AI

Let's use a language model to provide personalized financial advice based on user's financial situation and goals.


In [ ]:
# Financial situation parameters
age_slider = widgets.IntSlider(
    value=35, min=18, max=80, step=1, description="Age:", style={"description_width": "initial"}
)

income_slider = widgets.IntSlider(
    value=75000,
    min=20000,
    max=200000,
    step=5000,
    description="Annual Income ($):",
    style={"description_width": "initial"},
)

savings_slider = widgets.IntSlider(
    value=30000,
    min=0,
    max=500000,
    step=5000,
    description="Current Savings ($):",
    style={"description_width": "initial"},
)

debt_slider = widgets.IntSlider(
    value=15000,
    min=0,
    max=200000,
    step=5000,
    description="Total Debt ($):",
    style={"description_width": "initial"},
)

risk_tolerance = widgets.RadioButtons(
    options=["Low", "Medium", "High"],
    value="Medium",
    description="Risk Tolerance:",
    style={"description_width": "initial"},
)

financial_goals = widgets.SelectMultiple(
    options=[
        "Retirement",
        "Home Purchase",
        "Education Funding",
        "Debt Repayment",
        "Emergency Fund",
        "Wealth Building",
    ],
    value=["Retirement", "Emergency Fund"],
    description="Financial Goals:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="50%"),
)

advice_level = widgets.RadioButtons(
    options=["Beginner", "Intermediate", "Advanced"],
    value="Intermediate",
    description="Knowledge Level:",
    style={"description_width": "initial"},
)

get_advice_button = widgets.Button(
    description="Get Personalized Advice",
    button_style="primary",
    layout=widgets.Layout(width="200px"),
)

advice_output = widgets.Output()


def get_ml_advice(profile):
    """Generate advice using a rule-based system as fallback when OpenAI is not available"""
    age = profile["age"]
    income = profile["income"]
    savings = profile["savings"]
    debt = profile["debt"]
    risk = profile["risk_tolerance"]
    goals = profile["financial_goals"]
    knowledge = profile["knowledge_level"]

    # Calculate some financial ratios
    savings_to_income = savings / income if income > 0 else 0
    debt_to_income = debt / income if income > 0 else 0

    advice = []

    # Age-based advice
    if age < 30:
        stocks_allocation = 80 if risk == "High" else (70 if risk == "Medium" else 60)
        advice.append(
            f"At your age of {age}, you have a long investment horizon. Consider a {stocks_allocation}% allocation to stocks for long-term growth."
        )
    elif age < 50:
        stocks_allocation = 70 if risk == "High" else (60 if risk == "Medium" else 50)
        advice.append(
            f"At your age of {age}, you're in your peak earning years. Consider a {stocks_allocation}% allocation to stocks, balanced with fixed income."
        )
    else:
        stocks_allocation = 60 if risk == "High" else (50 if risk == "Medium" else 40)
        advice.append(
            f"At your age of {age}, as you approach retirement, consider a more conservative {stocks_allocation}% allocation to stocks."
        )

    # Savings advice
    if savings_to_income < 0.5:
        advice.append(
            f"Your savings of ${savings:,} is less than half your annual income. Consider increasing your emergency fund to 3-6 months of expenses."
        )
    else:
        advice.append(
            f"Your savings of ${savings:,} is {savings_to_income:.1f}x your annual income. This is a good foundation."
        )

    # Debt advice
    if debt_to_income > 0.4:
        advice.append(
            f"Your debt-to-income ratio of {debt_to_income:.2f} is high. Consider prioritizing debt reduction."
        )
    elif debt_to_income > 0.2:
        advice.append(
            f"Your debt-to-income ratio of {debt_to_income:.2f} is moderate. Balance debt repayment with investing."
        )
    else:
        advice.append(
            f"Your debt-to-income ratio of {debt_to_income:.2f} is relatively low. You're in a good position to focus on investing."
        )

    # Goal-specific advice
    if "Retirement" in goals:
        if age < 40:
            advice.append(
                "For retirement, maximize contributions to tax-advantaged accounts like 401(k) and IRAs."
            )
        else:
            advice.append(
                "For retirement, consider catch-up contributions and gradually shifting to more conservative investments."
            )

    if "Home Purchase" in goals:
        advice.append(
            "For a home purchase, consider keeping your down payment funds in a high-yield savings account or short-term CDs."
        )

    if "Education Funding" in goals:
        advice.append(
            "For education funding, consider 529 plans for tax-advantaged education savings."
        )

    # Format based on knowledge level
    if knowledge == "Beginner":
        return "\n\n".join(advice)
    elif knowledge == "Intermediate":
        # Add some more technical terms for intermediate level
        advanced_points = [
            f"Consider a portfolio with a Sharpe ratio optimized for your risk profile of {risk}.",
            "Review expense ratios on your investments to minimize drag on returns.",
        ]
        return "\n\n".join(advice + advanced_points)
    else:  # Advanced
        # Add even more technical concepts for advanced level
        advanced_points = [
            f"Implement a factor-based investment approach with tilts toward value, quality, and momentum based on your {risk} risk tolerance.",
            "Consider tax-loss harvesting strategies to optimize after-tax returns.",
            "Evaluate the use of options strategies for hedging or income generation based on market conditions.",
        ]
        return "\n\n".join(advice + advanced_points)


def get_llm_advice(profile):
    """Generate advice using OpenAI"""
    if not has_openai_key:
        return get_ml_advice(profile)  # Fallback to rule-based system

    try:
        # Format the financial profile for the prompt
        goals_str = ", ".join(profile["financial_goals"])

        prompt = f"""You are a personalized financial advisor. Provide specific financial advice for a user with the following profile:
        
        Age: {profile['age']}
        Annual Income: ${profile['income']:,}
        Current Savings: ${profile['savings']:,}
        Total Debt: ${profile['debt']:,}
        Risk Tolerance: {profile['risk_tolerance']}
        Financial Goals: {goals_str}
        
        The user has a {profile['knowledge_level'].lower()} level of financial knowledge. 
        Tailor your response to this knowledge level, using appropriate terminology and concepts.
        
        Provide specific, actionable advice in these areas:
        1. Asset allocation and investment strategy
        2. Savings recommendations
        3. Debt management
        4. Specific advice for each of their financial goals
        5. Tax considerations
        
        Your response should be well-structured and specifically tailored to their situation."""

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "You are a financial advisor AI that provides personalized financial advice.",
                },
                {"role": "user", "content": prompt},
            ],
            max_tokens=800,
            temperature=0.7,
        )

        return response.choices[0].message.content
    except Exception as e:
        print(f"Error with OpenAI API: {e}")
        return get_ml_advice(profile)  # Fallback to rule-based system


def generate_advice(b):
    with advice_output:
        clear_output()
        display(
            HTML("<p>Analyzing your financial profile and generating personalized advice...</p>")
        )

        # Create financial profile from inputs
        profile = {
            "age": age_slider.value,
            "income": income_slider.value,
            "savings": savings_slider.value,
            "debt": debt_slider.value,
            "risk_tolerance": risk_tolerance.value,
            "financial_goals": financial_goals.value,
            "knowledge_level": advice_level.value,
        }

        # Machine learning part: Classify the user's financial profile
        # Create a feature vector
        feature_vector = np.array(
            [
                [
                    profile["age"],
                    profile["income"],
                    profile["savings"],
                    profile["debt"],
                    (
                        1
                        if profile["risk_tolerance"] == "Low"
                        else (2 if profile["risk_tolerance"] == "Medium" else 3)
                    ),
                    len(profile["financial_goals"]),
                ]
            ]
        )

        # Normalize features
        scaler = StandardScaler()
        normalized_vector = scaler.fit_transform(feature_vector)

        # Use KMeans to classify the profile (in real application, this would be trained on more data)
        kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
        cluster = kmeans.fit_predict(normalized_vector)[0]

        # Map cluster to a profile type
        profile_types = ["Conservative Saver", "Balanced Investor", "Growth-Focused Investor"]
        profile_type = profile_types[cluster]

        clear_output()

        # Display financial profile summary
        display(HTML(f"<h3>Your Financial Profile</h3>"))
        display(HTML(f"<p><b>Age:</b> {profile['age']} years</p>"))
        display(HTML(f"<p><b>Annual Income:</b> ${profile['income']:,}</p>"))
        display(HTML(f"<p><b>Current Savings:</b> ${profile['savings']:,}</p>"))
        display(HTML(f"<p><b>Total Debt:</b> ${profile['debt']:,}</p>"))
        display(HTML(f"<p><b>Risk Tolerance:</b> {profile['risk_tolerance']}</p>"))
        display(HTML(f"<p><b>Financial Goals:</b> {', '.join(profile['financial_goals'])}</p>"))
        display(HTML(f"<p><b>Knowledge Level:</b> {profile['knowledge_level']}</p>"))
        display(HTML(f"<p><b>AI-Determined Profile Type:</b> {profile_type}</p>"))

        # Display personalized advice
        display(HTML(f"<h3>Personalized Financial Advice</h3>"))
        advice = get_llm_advice(profile)
        display(Markdown(advice))


get_advice_button.on_click(generate_advice)

# Display the widgets
display(widgets.HTML("<h2>AI-Powered Financial Advisor</h2>"))
display(widgets.HTML("<p>Enter your financial information to receive personalized advice:</p>"))
display(
    widgets.VBox(
        [
            widgets.HBox(
                [
                    widgets.VBox([age_slider, income_slider, savings_slider, debt_slider]),
                    widgets.VBox([risk_tolerance, financial_goals, advice_level]),
                ]
            ),
            get_advice_button,
            advice_output,
        ]
    )
)

HTML(value='<h2>AI-Powered Financial Advisor</h2>')

HTML(value='<p>Enter your financial information to receive personalized advice:</p>')

## Machine Learning-Based Investment Risk Assessment

Let's demonstrate how machine learning can be used to assess investment risk based on various factors.


In [11]:
# Create synthetic data for demonstration
np.random.seed(42)
n_samples = 1000

# Generate synthetic features
market_volatility = np.random.uniform(5, 30, n_samples)  # VIX index-like
price_to_earnings = np.random.uniform(10, 40, n_samples)  # P/E ratio
debt_to_equity = np.random.uniform(0.1, 2.0, n_samples)  # D/E ratio
dividend_yield = np.random.uniform(0, 7, n_samples)  # Dividend yield %
market_cap = np.random.uniform(1, 100, n_samples)  # In billions
interest_rate_environment = np.random.uniform(0, 8, n_samples)  # Interest rate %

# Create risk function (simplified for demonstration)
risk_score = (
    0.3 * market_volatility
    + 0.2 * price_to_earnings
    + 0.25 * debt_to_equity
    - 0.15 * dividend_yield
    - 0.1 * np.log(market_cap)
    + 0.2 * interest_rate_environment
)

# Normalize risk score
risk_score = (risk_score - np.min(risk_score)) / (np.max(risk_score) - np.min(risk_score)) * 100

# Create risk categories
risk_category = [
    "Low" if score < 33 else "Medium" if score < 66 else "High" for score in risk_score
]

# Create DataFrame
investment_data = pd.DataFrame(
    {
        "Market_Volatility": market_volatility,
        "Price_to_Earnings": price_to_earnings,
        "Debt_to_Equity": debt_to_equity,
        "Dividend_Yield": dividend_yield,
        "Market_Cap": market_cap,
        "Interest_Rate": interest_rate_environment,
        "Risk_Score": risk_score,
        "Risk_Category": risk_category,
    }
)

# Train a Random Forest Classifier
X = investment_data.drop(["Risk_Score", "Risk_Category"], axis=1)
y = investment_data["Risk_Category"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Feature importance
feature_importance = pd.DataFrame(
    {"Feature": X.columns, "Importance": model.feature_importances_}
).sort_values("Importance", ascending=False)

# Create widgets for user input
volatility_slider = widgets.FloatSlider(
    value=15,
    min=5,
    max=30,
    step=0.5,
    description="Market Volatility:",
    style={"description_width": "initial"},
)

pe_slider = widgets.FloatSlider(
    value=20,
    min=10,
    max=40,
    step=0.5,
    description="Price/Earnings:",
    style={"description_width": "initial"},
)

de_slider = widgets.FloatSlider(
    value=0.8,
    min=0.1,
    max=2.0,
    step=0.05,
    description="Debt/Equity:",
    style={"description_width": "initial"},
)

dividend_slider = widgets.FloatSlider(
    value=2.0,
    min=0,
    max=7,
    step=0.1,
    description="Dividend Yield (%):",
    style={"description_width": "initial"},
)

cap_slider = widgets.FloatSlider(
    value=20,
    min=1,
    max=100,
    step=1,
    description="Market Cap ($B):",
    style={"description_width": "initial"},
)

interest_slider = widgets.FloatSlider(
    value=3.0,
    min=0,
    max=8,
    step=0.25,
    description="Interest Rate (%):",
    style={"description_width": "initial"},
)

assess_button = widgets.Button(
    description="Assess Risk", button_style="primary", layout=widgets.Layout(width="200px")
)

risk_output = widgets.Output()


def assess_risk(b):
    with risk_output:
        clear_output()

        # Get input values
        input_features = pd.DataFrame(
            {
                "Market_Volatility": [volatility_slider.value],
                "Price_to_Earnings": [pe_slider.value],
                "Debt_to_Equity": [de_slider.value],
                "Dividend_Yield": [dividend_slider.value],
                "Market_Cap": [cap_slider.value],
                "Interest_Rate": [interest_slider.value],
            }
        )

        # Calculate risk score using the same formula
        risk_value = (
            0.3 * input_features["Market_Volatility"]
            + 0.2 * input_features["Price_to_Earnings"]
            + 0.25 * input_features["Debt_to_Equity"]
            - 0.15 * input_features["Dividend_Yield"]
            - 0.1 * np.log(input_features["Market_Cap"])
            + 0.2 * input_features["Interest_Rate"]
        )

        # Normalize using the same range as training data
        risk_min = np.min(risk_score)
        risk_max = np.max(risk_score)
        normalized_risk = (risk_value - risk_min) / (risk_max - risk_min) * 100
        normalized_risk = normalized_risk.values[0]

        # Use model to predict risk category
        predicted_category = model.predict(input_features)[0]

        # Get prediction probabilities
        prob = model.predict_proba(input_features)[0]
        prob_dict = {cat: prob[i] for i, cat in enumerate(model.classes_)}

        # Display results
        display(HTML(f"<h3>Investment Risk Assessment Results</h3>"))
        display(
            HTML(
                f"<p><b>Predicted Risk Category:</b> <span style='color: {'red' if predicted_category == 'High' else 'orange' if predicted_category == 'Medium' else 'green'}'>{predicted_category}</span></p>"
            )
        )
        display(HTML(f"<p><b>Calculated Risk Score:</b> {normalized_risk:.2f}/100</p>"))

        # Display probability breakdown
        display(HTML("<p><b>Probability Breakdown:</b></p>"))

        # Create probability bar chart
        fig = go.Figure()
        categories = list(prob_dict.keys())
        probabilities = list(prob_dict.values())
        colors = [
            "green" if cat == "Low" else "orange" if cat == "Medium" else "red"
            for cat in categories
        ]
        fig.add_trace(
            go.Bar(
                x=categories,
                y=probabilities,
                marker_color=colors,
                text=[f"{p*100:.1f}%" for p in probabilities],
                textposition="auto",
            )
        )
        fig.update_layout(
            title="Risk Category Probabilities",
            yaxis=dict(title="Probability", range=[0, 1]),
            height=300,
        )
        fig.show()

        # Create gauge chart for risk score
        gauge_fig = go.Figure()
        gauge_fig.add_trace(
            go.Indicator(
                mode="gauge+number",
                value=normalized_risk,
                title={"text": "Risk Score"},
                gauge={
                    "axis": {"range": [0, 100]},
                    "bar": {"color": "rgba(0,0,0,0)"},  # Make the bar transparent
                    "steps": [
                        {"range": [0, 33], "color": "rgba(0, 200, 0, 0.4)"},
                        {"range": [33, 66], "color": "rgba(255, 165, 0, 0.4)"},
                        {"range": [66, 100], "color": "rgba(255, 0, 0, 0.4)"},
                    ],
                    "threshold": {
                        "line": {"color": "black", "width": 3},
                        "thickness": 0.75,
                        "value": normalized_risk,
                    },
                },
            )
        )
        gauge_fig.update_layout(height=250)
        gauge_fig.show()

        # Feature importance for this prediction
        # Calculate SHAP values or simpler version for this demo
        feature_impact = {
            "Market_Volatility": 0.3 * input_features["Market_Volatility"].values[0],
            "Price_to_Earnings": 0.2 * input_features["Price_to_Earnings"].values[0],
            "Debt_to_Equity": 0.25 * input_features["Debt_to_Equity"].values[0],
            "Dividend_Yield": -0.15 * input_features["Dividend_Yield"].values[0],
            "Market_Cap": -0.1 * np.log(input_features["Market_Cap"].values[0]),
            "Interest_Rate": 0.2 * input_features["Interest_Rate"].values[0],
        }

        # Normalize impact values
        total_impact = sum(abs(v) for v in feature_impact.values())
        normalized_impact = {k: abs(v) / total_impact for k, v in feature_impact.items()}

        # Sort by absolute impact
        sorted_impact = sorted(normalized_impact.items(), key=lambda x: x[1], reverse=True)

        display(HTML("<p><b>Top Factors Influencing Risk:</b></p>"))
        fig_impact = go.Figure()
        fig_impact.add_trace(
            go.Bar(
                x=[f[0] for f in sorted_impact],
                y=[f[1] for f in sorted_impact],
                marker_color="steelblue",
                text=[f"{f[1]*100:.1f}%" for f in sorted_impact],
                textposition="auto",
            )
        )
        fig_impact.update_layout(title="Feature Impact on Risk Assessment", height=300)
        fig_impact.show()

        # Investment recommendation based on risk
        if predicted_category == "Low":
            recommendation = "This investment appears to have a low risk profile. It may be suitable for conservative investors or as part of a diversified portfolio."
        elif predicted_category == "Medium":
            recommendation = "This investment has a moderate risk profile. Consider balancing it with lower-risk assets and ensure it aligns with your overall risk tolerance."
        else:  # High
            recommendation = "This investment has a high risk profile. It may be appropriate for investors with high risk tolerance and long time horizons. Consider limiting exposure as part of a diversified portfolio."

        display(HTML(f"<p><b>Investment Recommendation:</b> {recommendation}</p>"))


assess_button.on_click(assess_risk)

# Display model performance
display(widgets.HTML(f"<h2>ML-Based Investment Risk Assessment</h2>"))
display(widgets.HTML(f"<p><i>Model accuracy on test data: {accuracy:.2%}</i></p>"))
display(widgets.HTML("<p>Adjust the parameters below to assess investment risk:</p>"))

# Create feature importance visualization
fi_fig = px.bar(
    feature_importance,
    x="Importance",
    y="Feature",
    orientation="h",
    title="Feature Importance in Risk Prediction Model",
    color="Importance",
    color_continuous_scale="Viridis",
)
fi_fig.update_layout(height=300)
fi_fig.show()

# Display widgets
display(
    widgets.VBox(
        [
            widgets.HTML("<p><b>Enter Investment Parameters:</b></p>"),
            widgets.HBox(
                [
                    widgets.VBox([volatility_slider, pe_slider, de_slider]),
                    widgets.VBox([dividend_slider, cap_slider, interest_slider]),
                ]
            ),
            assess_button,
            risk_output,
        ]
    )
)

# Initial assessment
assess_risk(None)

HTML(value='<h2>ML-Based Investment Risk Assessment</h2>')

HTML(value='<p><i>Model accuracy on test data: 93.00%</i></p>')

HTML(value='<p>Adjust the parameters below to assess investment risk:</p>')

## Key Takeaways

1. **AI-powered financial advising** provides personalized recommendations based on individual circumstances and knowledge levels.

2. **Machine learning models** can assess investment risk by analyzing multiple factors simultaneously.

3. **Natural language processing** enables sentiment analysis of financial news and market information.

4. **Personalized explanations** at different knowledge levels make complex financial concepts more accessible.

5. **Interactive visualizations** combined with AI insights help users make more informed financial decisions.

6. **Automated risk assessment** tools can quickly evaluate investments based on key metrics and provide actionable recommendations.
